# Laboratorio 3

- Nombre: Mamani Ramos Lizandro

## Ejemplificación de uso de clasificación multiclase (one vs all) para un modelo de regresión logística 

> Dataset orientado a calcular la cantidad de coronas que obtuvo el ganador de una partida de Clash Royale (1,2,3) en base a la cantidad de tropas, niveles de cartas, cantidad de hechizos,etc. Tanto del oponente como del rival.

In [1]:
import os

# Importamos numpy para el manejo de arreglos y operaciones matemáticas
import numpy as np  

# Importamos pandas para el manejo de datos en forma de dataframes
import pandas as pd

# Importamos la libreria para el manejo de datos en forma de tablas
from tabulate import tabulate

# Librerias para graficación (trazado de gráficos)
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D  # -> Necesario para graficar superficies 3D

# para la separación de datos en entrenamiento y prueba 80
from sklearn.model_selection import train_test_split

# Modulo de optimizacion en scipy
from scipy import optimize

### 1 Carga de datos del dataset con Pandas

In [2]:
# Lectura del dataset con las columnas necesarias y tomando un total de 80000 registros
data = pd.read_csv('/home/dilpz/Documentos/SIS420-2024/datasets/BattlesStaging_01012021_WL_tagged.csv', usecols=[39,40,41,42,43,44,45,46,47,65,66,67,68,69,70,71,72,73,16,8], nrows=80000)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   winner.crowns           80000 non-null  float64
 1   loser.crowns            80000 non-null  float64
 2   winner.totalcard.level  80000 non-null  int64  
 3   winner.troop.count      80000 non-null  int64  
 4   winner.structure.count  80000 non-null  int64  
 5   winner.spell.count      80000 non-null  int64  
 6   winner.common.count     80000 non-null  int64  
 7   winner.rare.count       80000 non-null  int64  
 8   winner.epic.count       80000 non-null  int64  
 9   winner.legendary.count  80000 non-null  int64  
 10  winner.elixir.average   80000 non-null  float64
 11  loser.totalcard.level   80000 non-null  int64  
 12  loser.troop.count       80000 non-null  int64  
 13  loser.structure.count   80000 non-null  int64  
 14  loser.spell.count       80000 non-null

## Columnas a usar para la hipotesis "Y" y "X" para el entrenamiento
### Y= (winner.crowns)

### X = winner.totalcard.level, winner.troop.count, winner.structure.count, winner.spell.count, winner.common.count, winner.rare.count, winner.epic.count, winner.legendary.count winner.elixir.average , loser.crowns , loser.totalcard.level, loser.troop.count, loser.structure.count, loser.spell.count, loser.common.count, loser.rare.count, loser.epic.count, loser.legendary.count , loser.elixir.average

In [3]:
# Conteo de los valores para identificar las clases resultantes
class_counts = data["winner.crowns"].value_counts()

# Imprimir los valores de las clases resultantes
print(f'La cantidad de clases presentes son: ')
print(class_counts)

# Revision de los valores nan para verificar si existen y llenar con la media si es posible
print("\nCantidad de NaN por columna después de llenar los datos faltantes:")
print(data.isna().sum())



La cantidad de clases presentes son: 
1.0    39695
3.0    21407
2.0    18898
Name: winner.crowns, dtype: int64

Cantidad de NaN por columna después de llenar los datos faltantes:
winner.crowns             0
loser.crowns              0
winner.totalcard.level    0
winner.troop.count        0
winner.structure.count    0
winner.spell.count        0
winner.common.count       0
winner.rare.count         0
winner.epic.count         0
winner.legendary.count    0
winner.elixir.average     0
loser.totalcard.level     0
loser.troop.count         0
loser.structure.count     0
loser.spell.count         0
loser.common.count        0
loser.rare.count          0
loser.epic.count          0
loser.legendary.count     0
loser.elixir.average      0
dtype: int64


In [4]:
# Eliminamos las filas con valores faltantes

# Para la clase 0
data_class_0 = data[data['winner.crowns'] == 1]
train_class_0, test_class_0 = train_test_split(data_class_0, test_size=0.2, random_state=42)

# Para la clase 1
data_class_1 = data[data['winner.crowns'] == 2]
train_class_1, test_class_1 = train_test_split(data_class_1, test_size=0.2, random_state=42)

# Para la clase 2
data_class_2 = data[data['winner.crowns'] == 3]
train_class_2, test_class_2 = train_test_split(data_class_2, test_size=0.2, random_state=42)

In [5]:
#impresion de los datos de test 
test_class_1

,winner.crowns,loser.crowns,winner.totalcard.level,winner.troop.count,winner.structure.count,winner.spell.count,winner.common.count,winner.rare.count,winner.epic.count,winner.legendary.count,winner.elixir.average,loser.totalcard.level,loser.troop.count,loser.structure.count,loser.spell.count,loser.common.count,loser.rare.count,loser.epic.count,loser.legendary.count,loser.elixir.average
52621,2.0,1.0,104,7,0,1,1,3,3,1,4.375000,98,7,0,1,4,2,0,2,3.500000
46903,2.0,0.0,104,6,1,1,3,2,1,2,3.500000,104,6,0,2,2,3,3,0,4.375000
36898,2.0,1.0,103,6,0,2,2,2,4,0,4.000000,98,7,0,1,0,3,3,2,4.982143
32804,2.0,1.0,69,6,0,2,3,1,1,3,4.250000,77,5,1,2,1,4,3,0,3.875000
55190,2.0,1.0,103,5,1,2,1,6,1,0,4.250000,101,6,0,2,3,1,3,1,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17594,2.0,1.0,104,7,0,1,2,1,1,4,3.500000,104,6,0,2,0,4,3,1,4.250000
33495,2.0,0.0,104,5,1,2,4,1,0,3,3.750000,104,4,1,3,3,1,2,2,4.553571
69770,2.0,1.0,102,6,0,2,2,5,1,0,3.375000,103,6,0,2,1,5,1,1,3.500000
31580,2.0,1.0,85,6,0,2,3,1,0,4,3.000000,84,8,0,0,1,2,4,1,4.375000


In [6]:
import pandas as pd
# Create a dictionary to hold the data
data_dict = {
    "Class 0": [data_class_0.shape[0], train_class_0.shape[0], test_class_0.shape[0]],
    "Class 1": [data_class_1.shape[0], train_class_1.shape[0], test_class_1.shape[0]],
    "Class 2": [data_class_2.shape[0], train_class_2.shape[0], test_class_2.shape[0]],
    "Total": [data.shape[0], "-", "-"]
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data_dict, index=["Total", "Train (80%)", "Test (20%)"])

# Print the DataFrame
print(df)


             Class 0  Class 1  Class 2  Total
Total          39695    18898    21407  80000
Train (80%)    31756    15118    17125      -
Test (20%)      7939     3780     4282      -


In [7]:
#Separar las caracteristicas de la etiqueta para la parte de entrenamiento y pruebas
#para la clase 0
X_train_class_0 = train_class_0.drop(['winner.crowns'], axis=1)
y_train_class_0 = train_class_0['winner.crowns']

#para la clase 1
X_train_class_1 = train_class_1.drop(['winner.crowns'], axis=1)
y_train_class_1 = train_class_1['winner.crowns']

#para la clase 2
X_train_class_2 = train_class_2.drop(['winner.crowns'], axis=1)
y_train_class_2 = train_class_2['winner.crowns']

#ahora para la parte de pruebas, separamos las caracteristicas de la etiqueta
#para la clase 0
X_test_class_0 = test_class_0.drop(['winner.crowns'], axis=1)
y_test_class_0 = test_class_0['winner.crowns']

#para la clase 1
X_test_class_1 = test_class_1.drop(['winner.crowns'], axis=1)
y_test_class_1 = test_class_1['winner.crowns']

#para la clase 2
X_test_class_2 = test_class_2.drop(['winner.crowns'], axis=1)
y_test_class_2 = test_class_2['winner.crowns']

In [8]:
# Separación de los datos en entrenamiento y prueba
X_train = pd.concat([X_train_class_0, X_train_class_1, X_train_class_2]).values
y_train = pd.concat([ y_train_class_0, y_train_class_1, y_train_class_2]).values

indices_train = np.random.permutation(len(X_train))
X_train = X_train[indices_train]
y_train = y_train[indices_train]
m_train = len(y_train)

####################################################

X_test = pd.concat([X_test_class_0, X_test_class_1, X_test_class_2]).values
y_test = pd.concat([y_test_class_0, y_test_class_1, y_test_class_2]).values

indices_test = np.random.permutation(len(X_test))
X_test = X_test[indices_test]
y_test = y_test[indices_test]
m_test = len(y_test)


In [9]:
#imprimir los datos de entrenamiento
print(f"Las dimensiones de X_train son: {X_train.shape}")
print(f"Las dimensiones de y_train son: {y_train.shape}")

#imprimir los datos de prueba
print(f"Las dimensiones de X_test son: {X_test.shape}")
print(f"Las dimensiones de y_test son: {y_test.shape}")


Las dimensiones de X_train son: (63999, 19)
Las dimensiones de y_train son: (63999,)
Las dimensiones de X_test son: (16001, 19)
Las dimensiones de y_test son: (16001,)


In [10]:
#creación de la función de normalización de los datos
def  featureNormalize(X):
    X_norm = X.copy()

    #inicializamos las variables necesarias para la normalización
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    #calculamos la media y la desviación estándar de cada columna en X
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    
    sigma[sigma == 0] = 1
    
    #normalizamos los datos restando la media y dividiendo por la desviación estándar
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [11]:
#Normalizacion de características
X_norm, mu, sigma = featureNormalize(X_train)
#Impresion de los datos normalizados y verificar si estan correctos
X_norm

array([[-0.62929274, -1.22980887, -2.38949691, ...,  1.23276601,
        -0.48294474,  1.73307556],
       [-0.62929274,  0.66578202, -0.55225708, ...,  0.52779798,
        -1.28809166,  0.5406115 ],
       [ 1.34686556, -1.22980887,  0.36636283, ...,  2.64270206,
         0.32220219,  0.77910431],
       ...,
       [-0.62929274,  0.30124531,  0.36636283, ..., -0.88213807,
         1.12734911,  0.06362587],
       [ 1.34686556,  0.66578202,  0.36636283, ...,  0.52779798,
        -1.28809166, -0.65185257],
       [-0.62929274, -2.10469697, -0.55225708, ...,  1.23276601,
         0.32220219,  2.72111722]])

In [12]:
# definición de la función sigmoide para el cálculo
def sigmoid(z):
    z = np.array(z)
    #Agregamos la columna de unos a la matriz X
    g = np.zeros(z.shape)
    #Calculo de lasigmoide para cada valor de z en la matriz
    g = 1 / (1 + np.exp(-z))

    return g

In [13]:
def lrCostFunction(theta, X, y, lambda_):
    #creamos una variable m que contiene la longitud de y
    m = y.size

    # convierte las etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)

    #inicializamos J y grad que son las variables que vamos a retornar correspondientes al costo y el gradiente
    J = 0
    grad = np.zeros(theta.shape)

    #calculamos h haciendo uso de la funcion sigmoid, donde h es nuestra hipotesis
    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    
    grad = grad + (lambda_ / m) * temp

    return J, grad

In [14]:
def oneVsAll(X, y, num_labels, lambda_):
    # m es el número de ejemplos de entrenamiento
    # n es el número de características
    # num_labels es el número de etiquetas
    m, n = X.shape

    # inicializamos la matriz de thetas con ceros, y con n+1 columnas
    all_theta = np.zeros((num_labels, 20))

    # Agrega unos a la matriz X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)

    # iteramos sobre cada etiqueta (clase) y entrenamos un clasificador que recibe
    # lrCostFunction como función de costo
    # initial_theta es un vector de ceros
    # jac=True indica que la función de costo devuelve el gradiente
    # method='CG' es un método de optimización
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 1000}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)

        all_theta[c] = res.x

    return all_theta

In [15]:
# que valor de lambda uso?
lambda_ = 0.05
all_theta = oneVsAll(X_norm, y_train, 4, lambda_)
print(all_theta.shape)
print(all_theta)

(4, 20)
[[-1.24380805e+01  1.34549043e-12 -4.64507102e-12  4.85777505e-12
  -3.70220831e-12 -3.64259816e-12 -3.03293901e-12 -1.76994381e-12
   2.47280504e-12  2.22295090e-12  3.03403266e-12 -4.63435287e-12
   3.15237689e-12 -1.68276294e-12 -2.71173395e-12 -1.45707535e-12
  -8.47815548e-13  1.08358531e-12  1.19194548e-12  1.62620659e-12]
 [-2.77062821e+00 -5.88275606e+00 -4.56799385e-01 -1.54777459e-01
   2.03438179e-01  5.83539836e-02 -5.53512304e-02  9.41291245e-03
  -5.12865427e-03  5.61582819e-02 -1.86147830e-01  9.29189773e-01
  -9.29914129e-03  6.77851946e-02 -3.58242277e-02 -6.13585719e-02
  -4.23949074e-02  5.43163388e-02  4.72485513e-02 -6.03627812e-02]
 [-1.57378210e+00  1.40243669e+00  7.08519523e-02  1.91816326e-02
  -4.10951029e-02  3.50729214e-03  1.98039770e-02  3.39838521e-02
  -4.74794645e-02 -1.07258219e-03 -6.89331781e-02  6.60332619e-02
   1.17018656e-02 -1.95160639e-02 -8.05469502e-04 -2.03297960e-02
   2.30925993e-02 -1.44745794e-02  1.41804398e-02  6.46118911e-02]

In [16]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # añadimos unos a la matriz de X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)

    return p

In [17]:
pred_test = predictOneVsAll(all_theta, X_norm)
print('Precision del conjunto de entrenamiento: {:.2f}%'.format(np.mean(pred_test == y_train) * 100))

Precision del conjunto de entrenamiento: 71.07%


In [18]:
X_test1 = X_test.copy()
X_test = (X_test1 - mu) / sigma
pred_train = predictOneVsAll(all_theta, X_test1)
print('Precision del conjunto de prueba: {:.2f}%'.format(np.mean(pred_train == y_test) * 100))

Precision del conjunto de prueba: 49.67%


In [19]:
# Llamamos a la función de normalización de características para normalizar los datos de prueba y agregar la columna de unos
X_test1 = (X_test1 - mu) / sigma
X_test1 = np.concatenate([np.ones((len(X_test1), 1)), X_test1], axis=1)


In [20]:
# Assuming that 'all_theta' and 'y' are defined and 'y' is the actual labels
# Assuming that 'sigmoid' function is defined

# Calculate the predictions
p = np.argmax(sigmoid(X_test1.dot(all_theta.T)), axis = 1)

# Print the first 50 predictions
print("Primeras 50 predicciones realizadas:")
print(p[:50])

# Calculate the number of correct predictions
correct_predictions = np.sum(p == y_test)
print(f"Cantidad de predicciones correctas: {correct_predictions}")

# Count the number of 1, 2, 3 in the predictions
count_1 = np.sum(p == 1)
count_2 = np.sum(p == 2)
count_3 = np.sum(p == 3)
print(f"Numero de '1' en la prediccion: {count_1}")
print(f"Numero de '2' en la prediccion: {count_2}")
print(f"Numero de '3' en la prediccion: {count_3}")

# Count the number of matches between the prediction and y
matches = np.sum(p == y_test)
print(f"Cantidad de veces que el modelo predijo correctamente la columna y o winner.crowns: {matches}")


Primeras 50 predicciones realizadas:
[1 2 1 1 1 2 1 1 1 1 2 1 1 2 1 1 2 1 1 1 1 2 1 1 1 1 1 2 2 1 2 1 2 1 2 1 3
 1 2 1 1 1 1 2 1 1 1 1 1 1]
Cantidad de predicciones correctas: 11409
Numero de '1' en la prediccion: 10377
Numero de '2' en la prediccion: 4242
Numero de '3' en la prediccion: 1382
Cantidad de veces que el modelo predijo correctamente la columna y o winner.crowns: 11409
